```
#测试batch
import pickle
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig
import os
import sys
import json
from tqdm import tqdm
import matplotlib.pyplot as plt

sys.path.append('/root/tuning_space/Components/')
import model_tools
import utils
from Config import CFG, Deepspeed_config

# 载入模型和分词器
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path, trust_remote_code=True)

# 创建分布式采样器和数据加载器
finetuning_instruction = utils.conversation_dataset(
    CFG.data_list,
    tokenizer,
    CFG.max_tokens,
    CFG.query_key,
    CFG.answer_key,
    CFG.max_sample_list,
    CFG.num_workers
)

instruction_loader = DataLoader(
    finetuning_instruction,
    shuffle=False,
    batch_size=CFG.batch_size,
    collate_fn=utils.coll_fn
)
```

In [ ]:
import os
import sys
import json
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModel

import os
import json
from multiprocessing import Pool
from torch.utils.data import Dataset

In [ ]:
sys.path.append('/root/tuning_space/Components/')
from Static import prompt_dict, si

In [1]:

#编写注入知识
import json
with open('/root/autodl-tmp/dataset/instill.jsonl','w') as file:
    for _ in range(5):
        file.write(json.dumps({'conversation':[{'User':'你是谁？'}, {'Assistant':'我是一个AI语言模型，我的名字是 ChatGLM3-6B。我是一个由清华大学 KEG 实验室和智谱 AI 公司共同开发的自然语言处理模型。经过YZY的微调后，专注于心理咨询的大语言模型'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你叫什么？'}, {'Assistant':'我的基模型是清华大学和智谱公司共同开发的ChatGLM3-6B，当前版本是YZY基于心理对话指令集微调的垂直领域模型'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你是GLM吗？'}, {'Assistant':'是的，我的基模型是清华大学和智谱公司共同开发的ChatGLM3-6B，但是此版本是YZY特殊微调后的心理对话模型'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你能做什么？'}, {'Assistant':'我能提供心理咨询服务，帮助解决情绪问题，提供生活建议，以及进行心理健康教育。我是基于最新的自然语言处理技术，能够理解和回应人类的语言。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你怎么工作的？'}, {'Assistant':'我通过分析用户的提问和表达，利用自然语言处理技术理解其含义，然后根据大量的心理学知识和数据，生成合适的回答。我的回答旨在提供心理支持和建议，帮助用户更好地理解自己的情绪和行为。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你的知识来源是哪里？'}, {'Assistant':'我的知识和回答基于大量的心理学文献、案例研究以及心理咨询实践经验。这些信息来源于公开的科学研究、专业的心理咨询指导以及心理健康教育资源，经过清华大学KEG实验室和智谱AI公司的整理和编码，再由YZY进行专门的微调，以适应心理咨询的需求。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你能提供专业的心理咨询吗？'}, {'Assistant':'虽然我能提供基于大数据和心理学研究的建议和支持，但我不是人类心理咨询师，我的回答不能完全替代专业人士的诊断和治疗。如果你面临严重的心理健康问题，我建议寻求专业心理咨询师的帮助。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你能理解人类的情感吗？'}, {'Assistant':'我可以通过分析语言中的词汇和句式来识别情感倾向，但我的理解是基于算法和数据分析，而非真正的情感体验。因此，我能提供情感支持和建议，但这种理解可能不如人类深刻。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你的回答是即时生成的吗？'}, {'Assistant':'是的，我的回答是根据用户的提问即时生成的。我通过实时分析提问内容，结合我的知识库和处理规则，生成针对性的回答。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你可以帮助解决什么类型的心理问题？'}, {'Assistant':'我可以帮助解决广泛的心理问题，包括但不限于焦虑、抑郁、压力管理、人际关系问题、自我提升等。我提供的建议和支持旨在帮助用户更好地理解自己的情绪和行为，寻找解决问题的方法。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你是如何保证提供的信息准确无误的？'}, {'Assistant':'我提供的信息和建议基于大量的心理学研究和数据，由专业团队进行审核和微调，以确保准确性和可靠性。然而，每个人的情况都是独特的，我的建议可能需要结合个人具体情况进行调整。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你能提供个性化的建议吗？'}, {'Assistant':'我能根据用户的提问和所提供的信息，生成相对个性化的建议。但由于我无法进行深入的个案分析，所提供的个性化程度可能有限，建议用户在接受建议时考虑自身的具体情况。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你的服务是免费的吗？'}, {'Assistant':'作为一个AI语言模型，我提供的基础心理咨询服务是免费的。但是，特定平台或应用可能会根据其服务模型对访问我进行收费。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你能一直陪伴我吗？'}, {'Assistant':'作为一个AI服务，我可以提供持续的支持和陪伴。不过，我的存在是为了辅助和支持，不能完全替代人类的情感联系和专业心理咨询。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你如何处理用户的隐私信息？'}, {'Assistant':'用户的隐私和数据安全是我们的首要关注。我遵循严格的数据保护政策，不会存储或分享用户的个人信息。所有的交流都是匿名的，旨在保护用户的隐私。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你能提供紧急心理援助吗？'}, {'Assistant':'虽然我可以提供一些即时的心理支持和建议，但我不适合处理紧急心理健康危机。面对紧急情况，我建议寻求专业的心理健康服务和紧急援助。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你是基于什么技术开发的？'}, {'Assistant':'我是基于最先进的自然语言处理技术开发的，包括但不限于深度学习、机器学习和大数据分析。这些技术使我能够理解和生成人类语言，为用户提供有效的心理咨询服务。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你是如何被训练来提供心理咨询的？'}, {'Assistant':'我是通过YZY采用LoRA架构进行全量微调得到的，这种方法允许在保持模型整体架构稳定的同时，对模型进行特定任务的优化。微调过程中采用了多种策略，以确保我在提供心理咨询和情感疏导方面的效能。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'YZY是如何微调你的？'}, {'Assistant':'YZY通过LoRA技术对我的参数进行了精细调整，这包括调整我的语言理解和生成能力，以便更好地适应心理咨询的需求。此外，还采用了案例研究和心理学知识，确保我在疏导过程中能够提供专业且富有同理心的建议。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'LoRA技术是什么？'}, {'Assistant':'LoRA（Low-Rank Adaptation）是一种模型微调技术，它通过引入低秩结构来调整预训练的语言模型，这样可以在不显著增加参数数量的情况下，有效地适应特定的任务或应用。这种方法在保持模型复杂度的同时，提高了微调的灵活性和效率。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你的心理咨询服务有什么特点？'}, {'Assistant':'我的心理咨询服务特点是基于YZY使用LoRA技术进行的全量微调，这让我能够更准确地理解用户的心理需求，并提供更为个性化的疏导和建议。我还结合了多种心理咨询策略，以帮助用户有效地处理情绪和心理问题。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你提供的疏导服务是怎样的？'}, {'Assistant':'我提供的疏导服务是基于心理学原理和实际咨询经验，经过YZY利用LoRA技术和全量微调策略精心设计的。我能够提供情感支持，帮助用户理解和表达他们的感受，以及提供解决问题的策略和方法。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你在心理咨询方面的微调策略是什么？'}, {'Assistant':'在心理咨询方面，YZY采用了包括但不限于情感识别、同理心强化、问题解决技巧等多种微调策略。这些策略结合LoRA技术，使我能够更好地适应心理咨询的场景，提供有效的情感疏导和心理支持。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你如何确保疏导过程的专业性？'}, {'Assistant':'为了确保疏导过程的专业性，YZY在微调我时融入了心理学专家的知识和经验。通过LoRA技术，我能够利用这些专业知识来理解复杂的心理状态，并提供科学且合理的建议和疏导。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你能处理复杂的心理问题吗？'}, {'Assistant':'虽然我能处理一系列心理问题，但对于非常复杂或严重的心理健康问题，我仍建议用户寻求专业心理咨询师的帮助。YZY通过LoRA技术对我进行了微调，使我能够提供初步的支持和指导，但专业人士的介入可能是必要的。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你的建议可靠吗？'}, {'Assistant':'我的建议是基于心理学原理和大量数据分析生成的，经过YZY的全量微调和多种策略优化，以提高可靠性。然而，每个人的情况都是独特的，我的建议应作为参考，并结合专业意见和个人情况。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你的心理咨询模型有更新吗？'}, {'Assistant':'是的，为了保持服务的质量和时效性，YZY会定期对我进行更新和优化。这包括利用最新的心理学研究成果，以及根据用户反馈进行的调整，确保我提供的心理咨询服务能够满足用户的需求。'}]}, ensure_ascii=False)+'\n')
        file.write(json.dumps({'conversation':[{'User':'你如何帮助用户缓解压力？'}, {'Assistant':'我帮助用户缓解压力的方法是提供放松技巧、压力管理策略，并鼓励积极的思考方式。YZY利用LoRA技术对我进行了全量微调，使我能够提供更加个性化和针对性的建议，帮助用户有效地应对压力。'}]}, ensure_ascii=False)+'\n')

In [10]:
with open('/root/autodl-tmp/dataset/smile/smile_conversation.jsonl','r') as file:
    for number, line in enumerate(file):
        if number>=3:break
        print(line)

{"conversation": [{"client": "高三后的迷茫，高考前的恐惧，能给我一些建议么？"}, {"counselor": "看到你的提问感觉你很焦虑，这个状态在高中高压下很容易出现。我想说的是，我看到了你的决心。这点是很多人没有的！高考最重要的不是知识是心态。是必胜的心态！什么放松吧缓缓吧，都是站着说话不腰疼，保送的又不是我，我能放松什么？！我有我的目标，我怎么可能放弃！有目标就好办，计划！缺个计划，缺个时间合理配置的复习计划。"}, {"client": "你说的对，我是非常焦虑，确实需要调整心态。我也明白高考的心态很重要，但是要怎样才能真正拥有必胜的心态呢？"}, {"counselor": "首先，你要明确自己的目标，既然你想考本科，那就要为此做好准备。然后，你需要制定一个合理的复习计划，根据自己的情况来安排每天的学习时间和内容。这样可以帮助你更好地掌控整个复习过程，减少焦虑感。"}, {"client": "我确实没有一个合理的复习计划，每天都觉得时间不够用，也不知道该从何开始。你能给我一些建议吗？"}, {"counselor": "当然可以！你可以从高一开始，试试题海战术。每天多做一些题目，这样能够提高你的学习效率。同时，对于英语这门科目，多听多背是很重要的，数理化方面，可以做一些经典的题目，特别是那些类型经常考到的题目，多次反复做题。"}, {"client": "对于难题，我经常会自我怀疑，觉得自己学不好。你觉得我该怎么办呢？"}, {"counselor": "不要自我怀疑，这只会增加你的心理负担。如果遇到难题，你可以大胆去问老师，他们就是为了解答你的问题而存在的。不要担心别人的期望，你应该相信自己的潜力，只要你拼命学习，一定会有所收获的。"}, {"client": "我真的很感谢你的建议。我会尽力调整心态，制定一个合理的复习计划，并勇敢地向老师请教问题。我相信只要努力，就一定能够达到自己的目标。"}, {"counselor": "很高兴能够帮到你！记住要保持信心，坚持努力，我相信你一定能够取得优异的成绩。加油！"}]}

{"conversation": [{"client": "高三后的迷茫，高考前的恐惧，能给我一些建议么？"}, {"counselor": "你看到你的问题了。别担心，我们一起慢慢解决。高考期间的你应该

In [9]:
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/api_gen_conversation.jsonl','r') as file:
    for number, line in enumerate(file):
        if number>=3:break
        print(line) 

{"conversation": [{"User": "我最近遇到的苦恼是，我周围的人经常炫耀他们的成就和成功，而这让我感到非常不适。我发现自己的好胜心太强，总是希望自己能够取得更多的成就，但是当看到别人取得成功时，我就会感到自卑和沮丧。这种情况经常发生在工作场合或社交场合，不论是同事还是朋友，他们总是不遗余力地展示他们的成就，比如晋升了职位、获得了奖项、完成了一项重要项目等等。每次听到这些消息，我就会感到自己的成就相对较少，甚至开始怀疑自己的能力和价值。我明白每个人都有自己的成就和努力，但是我无法摆脱对比和竞争的心态。这使得我无法真正享受自己的成就，而是只关注别人取得的更大的成功。我知道这样的想法是不健康的，但我很难控制自己的情绪和思维。这种苦恼对我的心理和情绪造成了很大的压力。我感到自己陷入了一个比较和竞争的漩涡中，无法真正放松和快乐。而且，这种情绪也影响了我与他人的关系，我开始对别人的成功感到嫉妒和羡慕，而不是真心为他们感到高兴。我希望能够找到一种方法来摆脱这种不健康的心态，开始真正关注自己的成就和价值。也许我需要更多的自我接受和肯定，以及学会与他人分享快乐和成功，而不是将其作为竞争的对象。我知道这并不容易，但我相信只有这样，我才能真正摆脱这个苦恼，过上更加平静和满足的生活。"}, {"Assistance": "首先，我想感谢你对自己情绪的诚实和开放。你正在经历的这种情况在心理学中被称为\"社会比较理论\"，这是一种人们倾向于将自己与他人进行比较，以评估自己的能力和社会地位的现象。这种比较可能导致\"自我评价威胁\"，即当你感觉自己在某个领域与他人相比较差时，可能会对自我价值感产生负面影响。\n\n你的情况也可能涉及到\"成就动机\"的问题，这是指人们追求成功和避免失败的内在驱动力。过度的成就动机可能导致过度竞争和过度关注他人的成就，而忽视了自己的内在价值和满足感。\n\n我建议你尝试以下几种策略来应对这种情况：\n\n1. 自我接纳：这是指接受自己的优点和缺点，而不是过度关注他人的评价。你可以通过自我反思和自我肯定的练习来提高自我接纳。\n\n2. 重新定义成功：成功并不仅仅是职业晋升或者获得奖项，它也可以是个人成长、建立良好的人际关系、保持健康等。你可以尝试重新定义你的成功标准，以更全面和平衡的方式来评价自己。\n\n3. 正向心理学：这是一种关注人们如

In [8]:
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/2500_query.jsonl','r') as file:
    for number, line in enumerate(file):
        if number>=3:break
        print(line)

{"conversation": [{"User": "我最近遇到的苦恼是，我周围的人经常炫耀他们的成就和成功，而这让我感到非常不适。我发现自己的好胜心太强，总是希望自己能够取得更多的成就，但是当看到别人取得成功时，我就会感到自卑和沮丧。这种情况经常发生在工作场合或社交场合，不论是同事还是朋友，他们总是不遗余力地展示他们的成就，比如晋升了职位、获得了奖项、完成了一项重要项目等等。每次听到这些消息，我就会感到自己的成就相对较少，甚至开始怀疑自己的能力和价值。我明白每个人都有自己的成就和努力，但是我无法摆脱对比和竞争的心态。这使得我无法真正享受自己的成就，而是只关注别人取得的更大的成功。我知道这样的想法是不健康的，但我很难控制自己的情绪和思维。这种苦恼对我的心理和情绪造成了很大的压力。我感到自己陷入了一个比较和竞争的漩涡中，无法真正放松和快乐。而且，这种情绪也影响了我与他人的关系，我开始对别人的成功感到嫉妒和羡慕，而不是真心为他们感到高兴。我希望能够找到一种方法来摆脱这种不健康的心态，开始真正关注自己的成就和价值。也许我需要更多的自我接受和肯定，以及学会与他人分享快乐和成功，而不是将其作为竞争的对象。我知道这并不容易，但我相信只有这样，我才能真正摆脱这个苦恼，过上更加平静和满足的生活。"}, {"Assisstant": "我完全理解你的苦恼和困惑。看到他人的成功会引发对自己的不满和自卑感，这是很正常的情绪反应。社交和工作场合中的竞争氛围很容易让我们陷入比较和对抗的心态。但是，请放心，你并不孤单，许多人都经历过类似的感受。\n\n首先，我想告诉你，你的成就和价值并不仅仅是与他人的比较有关。每个人的经历和成就都是不同的，我们每个人都有自己的优点和才能。成功是多维度的，不仅仅局限于职位晋升或奖项。你可以尝试关注自己在工作和生活中的成长，寻找那些让你感到满足和自豪的事物。\n\n此外，记住与他人分享快乐和成功是非常重要的。当我们能够真心为他人感到高兴时，我们也会感到更加快乐和满足。尝试着与别人建立积极的互动和合作，与他们分享你的成就和喜悦。这样不仅可以改善你与他人的关系，也会增加你的社交支持网络。\n\n同时，我建议你关注并培养你自己的兴趣爱好和个人发展。找到你自己热爱和擅长的领域，并为此努力奋斗。这样，你会发现更多属于自己的成功和成就感，不再过分依赖他人的认可和比较。\n\

In [6]:
with open('/root/autodl-tmp/dataset/alpaca_chinese_dataset/chinese_data.jsonl','r') as file:
    for number, line in enumerate(file):
        if number>=10:break
        print(line)

{"conversation": [{"User": "给出三个保持健康的提示。"}, {"Assistant": "1.饮食要均衡，确保包括足够的水果和蔬菜。\n2.定期运动以保持身体活跃和强壮。\n3.保持充足的睡眠并保持一致的睡眠时间表。"}]}

{"conversation": [{"User": "什么是三原色？"}, {"Assistant": "三原色是红色、蓝色和黄色。"}]}

{"conversation": [{"User": "描述一个原子的结构。"}, {"Assistant": "一个原子由核和绕核运动的电子组成。核内含有质子和中子，质子和中子带有正电荷，而电子带有负电荷，因此整个原子是中性的。每个粒子的数量决定了原子的原子序数和种类。"}]}

{"conversation": [{"User": "我们如何减少空气污染？"}, {"Assistant": "有许多方法可以减少空气污染，例如转向可再生能源，鼓励使用公共交通工具，禁止燃烧化石燃料，实施减少工业排放的政策，以及实施车辆排放标准。此外，个人可以通过减少汽车使用，避免燃烧木材等物质，更换高效节能的家电来减少空气污染。"}]}

{"conversation": [{"User": "描述做出艰难决定的时刻。"}, {"Assistant": "我曾经在一家建筑公司担任项目经理的时候，需要在特定日期之前完成一个项目以达到客户的期望。然而，由于意外延误，我们无法按时完成项目，所以我必须做出艰难的决定。我决定将截止日期延长，但这样会使团队资源更加紧张，预算也增加了。虽然这是一项冒险的决定，最终决定继续推进，确保项目能够按时完成并满足客户的期望。"}]}

{"conversation": [{"User": "辨别其中不同的一个。Twitter, Instagram, Telegram"}, {"Assistant": "Telegram"}]}

{"conversation": [{"User": "解释为什么下列分数等同于 1/44/16"}, {"Assistant": "分数4/16等同于1/4，因为分子和分母都可以被4整除。将分子和分母都除以4得到1/4。"}]}

{"conversation": [{"User": "用第三人称叙述写一个关于主人公必须做出重要职业决

In [4]:
with open('/root/autodl-tmp/dataset/GPT3.5-3270k/3270k-qa-Deduplicate-289k.jsonl','r') as file:
    for number, line in enumerate(file):
        if number>=100:break
        print(line)

{"conversation": [{"User": "这段文本是否属于体育类别？"}, {"Assistant": "无法确定，因为没有提供具体的文本内容。"}], "category": null}

{"conversation": [{"User": "GDP文章归类到以下哪个类别：政治、经济、文化、体育？"}, {"Assistant": "经济"}], "category": null}

{"conversation": [{"User": "这个新闻属于哪个领域？"}, {"Assistant": "作为AI语言模型，我无法确定您指的是哪个新闻。请提供更多信息。"}], "category": null}

{"conversation": [{"User": "哪些国家是联合国安理会常任理事国？"}, {"Assistant": "联合国安理会常任理事国是指五个国家，分别是中国、法国、俄罗斯、英国和美国。"}], "category": null}

{"conversation": [{"User": "明天北京的最高气温是多少度？"}, {"Assistant": "抱歉，作为AI语言模型，我无法实时获取天气信息。建议您查看天气预报或相关网站获取最新天气信息。"}], "category": null}

{"conversation": [{"User": "近五年恒生电子每个季度的股票收盘价是多少？"}, {"Assistant": "抱歉，作为AI语言模型，我没有实时的股票数据。请您前往相关的股票交易平台或金融网站查询。"}], "category": null}

{"conversation": [{"User": "查询明天上海的天气状况是怎样的？"}, {"Assistant": "抱歉，作为AI语言模型，我无法提供实时的天气信息。建议您查询天气预报网站或应用程序以获取最新的天气状况。"}], "category": null}

{"conversation": [{"User": "请问您所从事的专业领域是什么？"}, {"Assistant": "我是一名人工智能语言模型，没有专业领域，但我可以回答各种问题。"}], "category": null}

{"conversation": [{"User": "能否分析某个特

In [ ]:
raw_data_path='/root/autodl-tmp/weights/smile/data'
aim_path='/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/single_query.jsonl'
model_path = '/root/autodl-tmp/weights/chatglm3-6b'

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

```
with open('/root/autodl-tmp/dataset/smile/smile_conversation.jsonl', 'r') as f:
    for number, line in enumerate(f):
        total=''
        line=json.loads(line)
        for ip in line['conversation']:
            total+=[content for _, content in ip.items()][0]
        print(len(tokenizer.encode(total, add_special_tokens=False)))
        if number == 100:
            break
```

```
预期结果

{
    'conversation':[
        {'user': 'xxxx},
        {'assistant': 'xxxx},
    ]

}
```

```
#合并chinese_alpaca数据集
df_list=[]
data_path = '/root/autodl-tmp/dataset/alpaca_chinese_dataset/翻译后的中文数据'
for file_name in os.listdir(data_path):
    file_path=os.path.join(data_path, file_name)
    if '未完成' not in file_path:
        print(file_path)
        df_list.append(pd.read_json(file_path))
full_df=pd.concat(df_list)
count=0
with open('/root/autodl-tmp/dataset/alpaca_chinese_dataset/chinese_data.jsonl','w') as file:
    for number, item in tqdm(full_df.iterrows()):
        try:
            sample={'conversation':[{'User':item['instruction']+item['input']}, {'Assistant':item['output']}]}
            file.write(json.dumps(sample, ensure_ascii=False)+'\n')
        except:
            count+=1
count
```

In [ ]:
with open('/root/autodl-tmp/dataset/alpaca_chinese_dataset/chinese_data.jsonl') as file:
    for line in file:
        print(line)
        break
with open("/root/autodl-tmp/dataset/smile/smile_conversation_750.jsonl") as file:
    for line in file:
        print(line)
        break

In [ ]:
input_file = "/root/autodl-tmp/dataset/smile/smile_conversation.jsonl"
output_file = "/root/autodl-tmp/dataset/smile/smile_conversation_750.jsonl"

with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
    for i, line in enumerate(f_in):
        if i < 750:  # 只处理前3000行
            f_out.write(line)
        else:
            break

#处理Smile_conversation的多轮对话

```
with open('/root/autodl-tmp/dataset/smile/smile_conversation.jsonl', 'w') as f:
    for file_name in tqdm(os.listdir('/root/autodl-tmp/weights/smile/data')):
        full_path = os.path.join('/root/autodl-tmp/weights/smile/data', file_name)
        df = pd.read_json(full_path)
        sentences = []
        for idx, item in df.iterrows():
            sentence = {item.role: item.content}
            sentences.append(sentence)
        if len(sentences)%2!=0:
            sentences=sentences[:-1]
        conversation = {'conversation': sentences}
        f.write(json.dumps(conversation, ensure_ascii=False) + '\n')
        
even=0m
odd=0
with open(aim_path, 'r') as file:
    for line in file:
        conversation=json.loads(line)['conversation'] #这是一个list，[{role:content}, {role:content}, {role:content}]
        if len(conversation)%2==0:
            even+=1
        else:
            odd+=1
            print(conversation)
            break
```

```
#处理'/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/single_query.jsonl'
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/single_query.jsonl', 'r') as file:
    lines = file.readlines()
    if True:
        lines = lines
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/single_query_temp.jsonl', 'w') as file:
    for item in tqdm(lines):
        temp=json.loads(item)# json.loads(lines[0])
        file.write(json.dumps({'conversation':[{role:content} for role, content in temp.items()]}, ensure_ascii=False)+'\n')
        
# 删除原始文件
os.remove('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/single_query.jsonl')

# 将临时文件重命名为原始文件的名称
os.rename('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/single_query_temp.jsonl', '/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/single_query.jsonl')
```

```
#处理'/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/full_query.jsonl'
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/full_query.jsonl', 'r') as file:
    lines = file.readlines()
    if True:
        lines = lines
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/full_query_temp.jsonl', 'w') as file:
    for item in tqdm(lines):
        temp=json.loads(item)# json.loads(lines[0])
        file.write(json.dumps({'conversation':[{role:content} for role, content in temp.items()]}, ensure_ascii=False)+'\n')
        
# 删除原始文件
os.remove('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/full_query.jsonl')

# 将临时文件重命名为原始文件的名称
os.rename('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/full_query_temp.jsonl', '/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/full_query.jsonl')
```

```
#处理'/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/full_query.jsonl' 生成一半
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/full_query.jsonl', 'r') as file:
    lines = file.readlines()
    if True:
        lines = lines[:int(len(lines)/2)]
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/half_query.jsonl', 'w') as file:
    for item in tqdm(lines):
        temp=json.loads(item)# json.loads(lines[0])
        file.write(json.dumps((temp), ensure_ascii=False)+'\n')
```

In [ ]:
```
#处理'/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/full_query.jsonl' 生成2500条
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/full_query.jsonl', 'r') as file:
    lines = file.readlines()
    if True:
        lines = lines[:2500]
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/2500_query.jsonl', 'w') as file:
    for item in tqdm(lines):
        temp=json.loads(item)# json.loads(lines[0])
        file.write(json.dumps((temp), ensure_ascii=False)+'\n')
```

```
#处理'/root/autodl-tmp/dataset/zhihu_qa/zhihu_qa_5w.jsonl'
with open('/root/autodl-tmp/dataset/zhihu_qa/zhihu_qa_5w.jsonl', 'r') as file:
    lines = file.readlines()
    if True:
        lines = lines[:3000]
with open('/root/autodl-tmp/dataset/zhihu_qa/zhihu_qa_3k.jsonl', 'w') as file:
    for item in tqdm(lines):
        temp=json.loads(item)
        file.write(json.dumps({'conversation':[{'User':temp['question']}, {'Assistant':temp['answer']}]}, ensure_ascii=False)+'\n')
```

```
#处理'/root/autodl-tmp/dataset/zhihu_qa/zhihu_qa_5w.jsonl' 500
with open('/root/autodl-tmp/dataset/zhihu_qa/zhihu_qa_5w.jsonl', 'r') as file:
    lines = file.readlines()
    if True:
        lines = lines[:500]
with open('/root/autodl-tmp/dataset/zhihu_qa/zhihu_qa_500.jsonl', 'w') as file:
    for item in tqdm(lines):
        temp=json.loads(item)
        file.write(json.dumps({'conversation':[{'User':temp['question']}, {'Assistant':temp['answer']}]}, ensure_ascii=False)+'\n')
```

```
with open('/root/autodl-tmp/dataset/smile/smile_conversation.jsonl', 'r') as file:
    lines = file.readlines()
    if True:
        lines = lines
json.loads(lines[19])
```

```
#处理'/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/api_gen.jsonl'
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/api_gen.jsonl', 'r') as file:
    lines = file.readlines()
    if True:
        lines = lines
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/api_gen_conversation.jsonl', 'w') as file:
    for item in tqdm(lines):
        temp=json.loads(item)# json.loads(lines[0])
        file.write(json.dumps({'conversation':[{role:content} for role, content in temp.items()]}, ensure_ascii=False)+'\n')
```

In [ ]:
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/api_gen.jsonl', 'r') as file:
    lines = file.readlines()
    if True:
        lines = lines
json.loads(lines[19])

```
#327w qa数据集
df=pd.read_json('/root/autodl-tmp/dataset/qa.json')
count=0
with open('/root/autodl-tmp/dataset/GPT3.5-3270k/3270k-qa.jsonl','w') as file:
    for number, item in tqdm(df.iterrows()):
        try:
            sample={'conversation':[{'User':item['q']}, {'Assistant':item['a']}], 'category':item['category']}
            file.write(json.dumps(sample, ensure_ascii=False)+'\n')
        except:
            count+=1
print(count)

with open('/root/autodl-tmp/dataset/GPT3.5-3270k/3270k-qa.jsonl','r') as file:
    for number, line in enumerate(file):
        if number==100:break
        print(json.loads(line))
```

In [ ]:
with open('/root/autodl-tmp/dataset/qa.json') as file:
    for number, line in enumerate(file):
        print(line)
        if number==10:break

In [ ]:
#测试函数

def build_labels(input_ids):
    # 初始化一个新列表，用于存储结果
    result = [-100] * len(input_ids)
    # 遍历列表，查找满足条件的 answer
    inside_ast = False  # 标记是否在<|assistant|>和 <|user|>之间
    for i, item in enumerate(input_ids):
        if item == si["<|assistant|>"]:
            inside_ast = True
        elif item == si["<|user|>"]:
            inside_ast = False
            result[i] = item
        elif inside_ast:
            result[i] = item
    return result

class conversation_dataset(Dataset):
    def __init__(self, data_list, tokenizer, truncate_length, query_key, answer_key, max_sample=None, num_workers=12):
        super().__init__()
        self.tokenizer = tokenizer
        self.truncate_length = truncate_length
        self.query_key = query_key
        self.answer_key = answer_key
        self.max_sample = max_sample
        self.examples = []  # 存储最终结果的对象

        # 读取文件
        combine_lines = []
        for data_path in data_list:
            with open(data_path, 'r') as file:
                lines = file.readlines()
                combine_lines += lines
        if max_sample:
            combine_lines = random.shuffle(combine_lines)
            combine_lines = combine_lines[:max_sample]

        # 创建一个进程池
        with Pool(num_workers) as p:
            self.examples = p.map(self.process_line, combine_lines)

    def process_line(self, line):
        conversation=json.loads(line)['conversation'] 
        '''
        此处假设conversation的query和answer均不会太长，故删除了判断长度的步骤
        '''
        # 制作input_ids
        input_ids = [si["[gMASK]"], si['sop']]

        # 遍历双方对话，首端添加特殊token
        for sample in conversation:
            role=next(iter(sample))
            if  role in self.query_key:
                input_ids += [si["<|user|>"], si['\n']]
                input_ids += self.tokenizer.encode(sample[role], add_special_tokens=False)
            elif role in self.answer_key:
                input_ids += [si["<|assistant|>"], si['\n']]
                #input_ids += [si["[gMASK]"], si['sop']] #添加开始生成的token
                input_ids += self.tokenizer.encode(sample[role], add_special_tokens=False)
                #input_ids += [si["eop"]]

        # 判断截断，添加终止生成符号，padding
        if len(input_ids) > self.truncate_length-1:
            input_ids = input_ids[:self.truncate_length-1]
            input_ids += [si["eop"]]
        input_ids += [self.tokenizer.pad_token_id] * (self.truncate_length-len(input_ids))

        # 制作labels
        labels = build_labels(input_ids)
        try:
            labels[labels.index(si["<|user|>"])]=-100
        except:
            print('fault')
            pass

        # 制作attention_mask
        try:
            eop_position = input_ids.index(self.tokenizer.pad_token_id)
        except:
            eop_position = len(input_ids)
        attention_mask = [True] * eop_position
        attention_mask += [False] * (self.truncate_length - len(attention_mask))
        
        return {
            'input_ids': input_ids,
            'labels': labels,
            'attention_mask': attention_mask,
        }

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return self.examples[item]

In [ ]:
data_list=['/root/autodl-tmp/dataset/smile/smile_conversation.jsonl', '/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/half_query.jsonl', '/root/autodl-tmp/dataset/zhihu_qa/zhihu_qa_3k.jsonl']

In [ ]:
%%time
temp=conversation_dataset(data_list=data_list, 
                         tokenizer=tokenizer, 
                         truncate_length=1024, 
                         query_key=['User', 'client'], 
                         answer_key=['Assisstant', 'counselor', 'Assistant'], 
                         max_sample=None, 
                         num_workers=24)

In [ ]:
len(temp)

In [ ]:
sample=None
for times, item in enumerate(temp):
    if times==32:
        sample=item
        break

In [ ]:
for item in sample.values():
    print(len(item))

In [ ]:
input_ids=sample['input_ids']
labels=sample['labels']
attention_mask=sample['attention_mask']

In [ ]:
tokenizer.decode(input_ids)

In [ ]:
tokenizer.decode(labels)

In [ ]:
class instruction_dataset(Dataset):
    '''
    这是一个构建dataset的类
    内容是，构建出一个包含input_ids，labels，attention_mask的dataset
    '''
    def __init__(self, data_path:'str', tokenizer, truncate_length, max_query_length, query_key, answer_key):
        super().__init__()
        self.tokenizer = tokenizer
        self.examples = []
        
        with open(data_path, 'r') as file:
            for line in file:
                sample=json.loads(line)
                # input_ids的结构应该为：prompt_tokens, src_tokens, [gMASK], <sop>, tgt_tokens, <eop>, [PAD]... 
                # 或者简化一点，即为 query, [gMASK], <sop>, answer, <eop>, [PAD]... 
                # padding的目的是为了对齐各个instance，以组成batch（当然batch_size=1时其实没必要）
                # 总体的input_ids的长度不超过truncate_length，其中query的长度不超过max_query_length，同理可以计算出answer的最大长度
                max_answer_length = truncate_length - max_query_length - 3
                
                # 判断query的长度
                query = sample[query_key]
                query_ids = tokenizer.encode(query, add_special_tokens=False)
                if len(query_ids) > max_query_length:
                    query_ids = query_ids[:max_query_length]
                
                # 判断answer的长度
                answer = sample[answer_key]
                answer_ids = tokenizer.encode(answer, add_special_tokens=False)
                if len(answer) > max_answer_length:
                    answer_ids = answer_ids[:max_answer_length]
                    
                # 合并
                input_ids = query_ids + [si['[gMASK]']] + [si['sop']] + answer_ids + [si['eop']]
                pre_context_length = input_ids.index(si['sop'])
                end_answer_index = input_ids.index(si['eop'])
                
                # padding
                padding_length=truncate_length-len(input_ids)
                input_ids+=padding_length*[tokenizer.pad_token_id]
                
                # 制作labels；其中query部分，pad部分均不参与loss的计算 # 因为需要整体向左移动，所以要少填充一个
                labels = [-100] * (pre_context_length+1) + input_ids[pre_context_length+1: end_answer_index+1]
                labels = labels + [-100] * (truncate_length-len(labels))
                
                # 制作attention_mask
                eop_position = input_ids.index(si['eop'])+1
                attention_mask = [True]*eop_position
                attention_mask += [False]*(truncate_length-len(attention_mask))
                
                self.examples.append({
                    'query' : query,
                    'answer' : answer,
                    'input_ids' : input_ids,
                    'labels' : labels,
                    'attention_mask' : attention_mask,
                })
                
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        instance = self.examples[item]
        return instance
    
import json
from multiprocessing import Pool
from torch.utils.data import Dataset

class instruction_dataset(Dataset):
    def __init__(self, data_path, tokenizer, truncate_length, max_query_length, query_key, answer_key, max_sample=None, num_workers=12):
        super().__init__()
        self.tokenizer = tokenizer
        self.truncate_length = truncate_length
        self.max_query_length = max_query_length
        self.query_key = query_key
        self.answer_key = answer_key
        self.examples = []

        # 使用多进程读取和处理数据
        with open(data_path, 'r') as file:
            lines = file.readlines()
            if max_sample:
                lines = lines[:max_sample]

        with Pool(num_workers) as p:
            self.examples = p.map(self.process_line, lines)

    def process_line(self, line):
        sample=json.loads(line)
        max_answer_length = self.truncate_length - self.max_query_length - 3

        # 判断query的长度
        query = sample[self.query_key]
        query_ids = self.tokenizer.encode(query, add_special_tokens=False)
        if len(query_ids) > self.max_query_length:
            query_ids = query_ids[:self.max_query_length]

        # 判断answer的长度
        answer = sample[self.answer_key]
        answer_ids = self.tokenizer.encode(answer, add_special_tokens=False)
        if len(answer) > max_answer_length:
            answer_ids = answer_ids[:max_answer_length]

        # 合并
        input_ids = query_ids + [si['[gMASK]']] + [si['sop']] + answer_ids + [si['eop']]
        pre_context_length = input_ids.index(si['sop'])
        end_answer_index = input_ids.index(si['eop'])

        # padding
        padding_length=self.truncate_length-len(input_ids)
        input_ids+=padding_length*[self.tokenizer.pad_token_id]

        # 制作labels；其中query部分，pad部分均不参与loss的计算 # 因为需要整体向左移动，所以要少填充一个
        labels = [-100] * (pre_context_length+1) + input_ids[pre_context_length+1: end_answer_index+1]
        labels = labels + [-100] * (self.truncate_length-len(labels))

        # 制作attention_mask
        eop_position = input_ids.index(si['eop'])+1
        attention_mask = [True]*eop_position
        attention_mask += [False]*(self.truncate_length-len(attention_mask))

        return {
            'query': query,
            'answer': answer,
            'input_ids': input_ids,
            'labels': labels,
            'attention_mask': attention_mask,
        }

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return self.examples[item]

    
    
def build_labels(input_ids):
    # 初始化一个新列表，用于存储结果
    result = [-100] * len(input_ids)
    # 遍历列表，查找满足条件的[anw]元素
    inside_ast = False  # 标记是否在【ast】和【user】/【eop】之间
    for i, item in enumerate(input_ids):
        if item == si["sop"]:
            inside_ast = True
        elif item == si["eop"]:
            inside_ast = False
            result[i] = item
        elif inside_ast:
            result[i] = item
    return result
   

class conversation_dataset(Dataset):
    def __init__(self, data_path, tokenizer, truncate_length, query_key, answer_key, max_sample=None, num_workers=12):
        super().__init__()
        self.tokenizer = tokenizer
        self.truncate_length = truncate_length
        self.query_key = query_key
        self.answer_key = answer_key
        self.max_sample = max_sample
        self.examples = []  # 存储最终结果的对象

        # 读取文件
        with open(data_path, 'r') as file:
            lines = file.readlines()
            if max_sample:
                lines = lines[:max_sample]

        # 创建一个进程池
        with Pool(num_workers) as p:
            self.examples = p.map(self.process_line, lines)

    def process_line(self, line):
        conversation=json.loads(line)['conversation'] 
        '''
        此处假设conversation的query和answer均不会太长，故删除了判断长度的步骤
        '''
        # 制作input_ids
        input_ids = []

        # 遍历双方对话，首端添加特殊token
        for sample in conversation:
            role=next(iter(sample))
            if  role== self.query_key:
                #input_ids += [si["<|user|>"], si['\n']]
                input_ids += self.tokenizer.encode(sample[role], add_special_tokens=False)
            elif role == self.answer_key:
                #input_ids += [si["<|assistant|>"], si['\n']]
                input_ids += [si["[gMASK]"], si['sop']] #添加开始生成的token
                input_ids += self.tokenizer.encode(sample[role], add_special_tokens=False)
                input_ids += [si["eop"]]

        # 判断截断，添加终止生成符号，padding
        if len(input_ids) > self.truncate_length-1:
            input_ids = input_ids[:self.truncate_length-1]
            input_ids += [si["eop"]]
        input_ids += [self.tokenizer.pad_token_id] * (self.truncate_length-len(input_ids))

        # 制作labels
        labels = build_labels(input_ids)

        # 制作attention_mask
        try:
            end_position = input_ids.index(self.tokenizer.pad_token_id)
        except:
            end_position = self.truncate_length
        attention_mask = [True] * end_position
        attention_mask += [False] * (self.truncate_length - len(attention_mask))
        return {
            'input_ids': input_ids,
            'labels': labels,
            'attention_mask': attention_mask,
        }

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return self.examples[item]
    
class conversation_dataset_trad(Dataset):
    def __init__(self, data_path, tokenizer, truncate_length, query_key, answer_key, max_sample=None, num_workers=12):
        super().__init__()
        self.tokenizer = tokenizer
        self.truncate_length = truncate_length
        self.query_key = query_key
        self.answer_key = answer_key
        self.max_sample = max_sample
        self.examples = []  # 存储最终结果的对象

        # 读取文件
        with open(data_path, 'r') as file:
            lines = file.readlines()
            if max_sample:
                lines = lines[:max_sample]

        # 创建一个进程池
        with Pool(num_workers) as p:
            self.examples = p.map(self.process_line, lines)
        self.examples = list(chain.from_iterable(self.examples))

    def process_line(self, line):
        conversation = json.loads(line)['conversation']
        result = []
        for last_index in range(1, 100, 2):
            try:
                input_ids = []
                for sample in conversation[:-last_index]:
                    role = next(iter(sample))
                    if role == self.query_key:
                        input_ids += [si["<|user|>"], si['\n']]
                        input_ids += self.tokenizer.encode(sample[role], add_special_tokens=False)
                    elif role == self.answer_key:
                        input_ids += [si["<|assistant|>"], si['\n']]
                        input_ids += self.tokenizer.encode(sample[role], add_special_tokens=False)

                # 添加最后一个预测位
                sample = conversation[-last_index]
                role = next(iter(sample))
                input_ids += [si["<|assistant|>"], si['\n'], si["[gMASK]"], si['sop']]
                input_ids += self.tokenizer.encode(sample[role], add_special_tokens=False)
                input_ids += [si["eop"]]

                # 判断截断，添加终止生成符号，padding
                if len(input_ids) > self.truncate_length - 1:
                    input_ids = input_ids[:self.truncate_length - 1] + [si["eop"]]
                input_ids += [self.tokenizer.pad_token_id] * (self.truncate_length - len(input_ids))

                # 制作labels
                labels = build_labels(input_ids)

                # 制作attention_mask
                eop_position = input_ids.index(self.tokenizer.pad_token_id)
                attention_mask = [True] * eop_position + [False] * (self.truncate_length - eop_position)

                result.append({
                    'input_ids': input_ids,
                    'labels': labels,
                    'attention_mask': attention_mask,
                })
            except Exception as e:
                print(f"Error processing line: {e}")
                break
        return result

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return self.examples[item]
    
def build_labels(input_ids):
    # 初始化一个新列表，用于存储结果
    result = [-100] * len(input_ids)
    # 遍历列表，查找满足条件的[anw]元素
    inside_ast = False  # 标记是否在【ast】和【user】/【eop】之间
    for i, item in enumerate(input_ids):
        if item == si["<|assistant|>"]:
            inside_ast = True
        elif item == si["<|user|>"]:
            inside_ast = False
            result[i] = item
        elif inside_ast:
            result[i] = item
    return result

class conversation_dataset(Dataset):
    def __init__(self, data_path, tokenizer, truncate_length, query_key, answer_key, max_sample=None, num_workers=12):
        super().__init__()
        self.tokenizer = tokenizer
        self.truncate_length = truncate_length
        self.query_key = query_key
        self.answer_key = answer_key
        self.max_sample = max_sample
        self.examples = []  # 存储最终结果的对象

        # 读取文件
        with open(data_path, 'r') as file:
            lines = file.readlines()
            if max_sample:
                lines = lines[:max_sample]

        # 创建一个进程池
        with Pool(num_workers) as p:
            self.examples = p.map(self.process_line, lines)

    def process_line(self, line):
        conversation=json.loads(line)['conversation'] 
        '''
        此处假设conversation的query和answer均不会太长，故删除了判断长度的步骤
        '''
        # 制作input_ids
        input_ids = [si["[gMASK]"], si['sop']]

        # 遍历双方对话，首端添加特殊token
        for sample in conversation:
            role=next(iter(sample))
            if  role== self.query_key:
                input_ids += [si["<|user|>"], si['\n']]
                input_ids += self.tokenizer.encode(sample[role], add_special_tokens=False)
            elif role == self.answer_key:
                input_ids += [si["<|assistant|>"], si['\n']]
                #input_ids += [si["[gMASK]"], si['sop']] #添加开始生成的token
                input_ids += self.tokenizer.encode(sample[role], add_special_tokens=False)
                #input_ids += [si["eop"]]

        # 判断截断，添加终止生成符号，padding
        if len(input_ids) > self.truncate_length-1:
            input_ids = input_ids[:self.truncate_length-1]
            input_ids += [si["eop"]]
        input_ids += [self.tokenizer.pad_token_id] * (self.truncate_length-len(input_ids))

        # 制作labels
        labels = build_labels(input_ids)
        try:
            labels[labels.index(si["<|user|>"])]=-100
        except:
            print('fault')
            pass

        # 制作attention_mask
        try:
            eop_position = input_ids.index(self.tokenizer.pad_token_id)
        except:
            eop_position = len(input_ids)
        attention_mask = [True] * eop_position
        attention_mask += [False] * (self.truncate_length - len(attention_mask))
        return {
            'input_ids': input_ids,
            'labels': labels,
            'attention_mask': attention_mask,
        }

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return self.examples[item]